In [4]:
__grader__="Emily Hua"

In [5]:
import pandas as pd
import numpy as np
from geopy.distance import vincenty
import sys

In [6]:
import warnings
warnings.filterwarnings("ignore")

In [7]:
'''
load the json submission file, store as list @submission
'''
import json

submission = []
with open('submission') as f:
    for line in f:
        submission.append(json.loads(line))

In [8]:
'''
@nodes stores the edges
'''
nodes = pd.read_csv("../usersGPS_larg_comp_index.txt", header = None, sep=' ')
nodes.columns = ['node1', 'node2']
nodes[:1]#preview

,node1,node2
0,0,1


In [9]:
'''
@G stores the undirected graph
'''
import networkx as nx
G = nx.from_pandas_dataframe(nodes, 'node1', 'node2')

In [11]:
'''
validate the path
'''
def is_valid (path, G):
    for index in range(len(path) - 1): 
        if (not(path[index] in G.neighbors(path[index + 1]))): #no an edge
            return False
    return True
            

In [12]:
'''
uni transformation
@uni_map
generate a list of uni
@uni_list
'''
from collections import defaultdict
uni_map = defaultdict(str)
uni_list = []
with open("new_uni.csv") as f:
    for line in f:
        line = line.rstrip('\n').split(",")
        uni_map[line[1]] = line[0]   
        uni_list.append(line[0])
        

In [17]:
'''
return the length of the path
'''
def find_length(path):
    return len(path)

In [18]:
'''load the checkin data'''
df_checkin = pd.read_csv("../checkin_usersGPS_largcomp_index.csv", header=None)

In [19]:
'''process the checkin data '''
df_checkin.columns = ['node', 'latitude', 'logitude','epoch']
df_checkin['node'] = df_checkin['node'].astype(int)
df_checkin['date'] = pd.to_datetime(df_checkin['epoch'],unit='s')
del df_checkin['epoch']
#preview first 2 entries
df_checkin[:2]

,node,latitude,logitude,date
0,6200,40.82942,-73.920529,2013-08-31 20:45:49
1,6200,40.82942,-73.920529,2013-08-31 20:42:34


In [20]:
'''to keep only the last checkin'''
idx = df_checkin.groupby(['node'])['date'].transform(max) == df_checkin['date']
#print (idx)
df_last_checkin = df_checkin[idx]

In [21]:
df_last_checkin['latitude'][df_last_checkin['node'] == 0].values[0]
df_last_checkin['logitude'][df_last_checkin['node'] == 0].values[0]

-74.097142738000002

In [23]:
'''
return the distance of the path
@get_distance can either return eculidean or geo distance
'''
def get_distance( path, df_checkin):
    dist_euclidean_sum = 0
    geo_dist_sum = 0
    for index in range(len(path) - 1 ):
        distances1 = []
        distances2 = []
        node1 = path[index]
        node2 = path[index+1]
        latitude1 = df_last_checkin['latitude'][df_last_checkin['node'] == node1].values[0]
        logitude1 = df_last_checkin['logitude'][df_last_checkin['node'] == node1].values[0]
        latitude2 = df_last_checkin['latitude'][df_last_checkin['node'] == node2].values[0]
        logitude2 = df_last_checkin['logitude'][df_last_checkin['node'] == node2].values[0]
        distances1.append(latitude1)
        distances1.append(logitude1)
        distances2.append(latitude2)
        distances2.append(logitude2)        
        distances1 = np.array([distances1])
        distances2 = np.array([distances2])

        eulidean_dist = np.linalg.norm(distances1 - distances2)
        dist_euclidean_sum += eulidean_dist       
        geo_dist = vincenty(distances1, distances2).miles
        geo_dist_sum += geo_dist
    return dist_euclidean_sum, geo_dist_sum
        
            

In [25]:
'''
create nested list to fromat submission the following way:
@table
testcase, uni, nOfQueries, path, isvalid, euclidean_dist, geo_dist
for testcase1-20
'''
table = []
counter = 0
for entry in submission:
    uni = uni_map[entry['uni']['s']]
    nOfQueries = entry['nOfQueries']['n']
    testcaseID = entry['testcaseID']['s']
    if (entry['path']['s'] == "null"):
        isvalid = -1
        path_length = -1
        euclidean_dist = -1
        geo_dist = -1
        
    else:
        path = [int(node) for node in entry['path']['s'].strip("\'").split(',')]
        isvalid = is_valid(path, G)
        path_length = find_length(path)
        euclidean_dist, geo_dist = get_distance(path, df_checkin)
    table.append([testcaseID, uni, nOfQueries, path, isvalid, path_length, euclidean_dist, geo_dist])
       
    if (counter == 800):
        uni = uni_map[entry['uni']['s']]
        print ("real uni: ", uni)
        print ("uni: ", entry['uni']['s'])
        nOfQueries = entry['nOfQueries']['n']

    counter += 1
    #print ("counter: ", counter)
    

real uni:  wh2353
uni:  flag


In [26]:
'''
convert nested list to panda dataframe
'''
df_table = pd.DataFrame(table)
df_table.columns = ['testcaseID', 'uni', 'nOfQueries', 'path', 'is_valid', 'path_length', 'euclidean_dist', 'geo_dist']

In [35]:
'''
generate total score dataframe based on nOfQueries manually

'''
for testcaseID in range(13, 14):
    table_by_testcase = df_table[df_table['testcaseID'] == str(testcaseID)]
    table_ready = process_table(table_by_testcase, 'nOfQueries')
    query_ranks = gen_rank(table_ready, 'nOfQueries')
    query_performance = gen_performance_score(query_ranks)
    query_total_score = gen_total_score(query_performance)
    query_total_score['testcaseID'] = testcaseID
    

base_on:  nOfQueries


In [55]:
'''
generate total score dataframe based on nOfQueries
@query_score
'''

from collections import defaultdict
query_score = defaultdict(float)
for testcaseID in range(1, 41):   
    print ("testcaseID: ", testcaseID)
    table_by_testcase = df_table[df_table['testcaseID'] == str(testcaseID)]
    table_ready = process_table(table_by_testcase, 'nOfQueries')
    query_ranks = gen_rank(table_ready, 'nOfQueries')
    query_performance = gen_performance_score(query_ranks)
    query_total_score = gen_total_score(query_performance)
    query_total_score['testcaseID'] = testcaseID
    if (testcaseID == 1):
        giant_query = query_total_score
    else:
        giant_query = [giant_query, query_total_score]
        giant_query = pd.concat(giant_query)
    for index, row in query_total_score.iterrows():
        key = frozenset({row['testcaseID'], row['uni']})
        query_score[key] = row['total_score']
    

testcaseID:  1
base_on:  nOfQueries
testcaseID:  2
base_on:  nOfQueries
testcaseID:  3
base_on:  nOfQueries
testcaseID:  4
base_on:  nOfQueries
testcaseID:  5
base_on:  nOfQueries
testcaseID:  6
base_on:  nOfQueries
testcaseID:  7
base_on:  nOfQueries
testcaseID:  8
base_on:  nOfQueries
testcaseID:  9
base_on:  nOfQueries
testcaseID:  10
base_on:  nOfQueries
testcaseID:  11
base_on:  nOfQueries
testcaseID:  12
base_on:  nOfQueries
testcaseID:  13
base_on:  nOfQueries
testcaseID:  14
base_on:  nOfQueries
testcaseID:  15
base_on:  nOfQueries
testcaseID:  16
base_on:  nOfQueries
testcaseID:  17
base_on:  nOfQueries
testcaseID:  18
base_on:  nOfQueries
testcaseID:  19
base_on:  nOfQueries
testcaseID:  20
base_on:  nOfQueries
testcaseID:  21
base_on:  nOfQueries
testcaseID:  22
base_on:  nOfQueries
testcaseID:  23
base_on:  nOfQueries
testcaseID:  24
base_on:  nOfQueries
testcaseID:  25
base_on:  nOfQueries
testcaseID:  26
base_on:  nOfQueries
testcaseID:  27
base_on:  nOfQueries
testcaseID

In [74]:
giant_query.columns = ['uni','query_rank', 'query_performance_score', 'query_lowest_rank', 'query_total_score', 'testcaseID']

In [75]:
giant_query.query_rank[giant_query['query_rank'] == 9223372036854775807 ] = 'invalid path'

In [77]:
giant_len.columns = ['uni', 'length_rank', 'length_performance_score', 'length_lowest_rank', 'length_total_score', 'testcaseID']
giant_len.length_rank[giant_len['length_rank'] == 9223372036854775807] = 'invalid path'

In [78]:
giant_geo.columns = ['uni', 'geo_rank', 'geo_performance_score', 'geo_lowest_rank', 'geo_total_score', 'testcaseID']
giant_geo.geo_rank[giant_geo['geo_rank'] == 9223372036854775807] = 'invalid path'

In [79]:
giant_eucli.columns = ['uni', 'eucli_rank', 'eucli_performance_score', 'eucli_lowest_rank', 'eucli_total_score', 'testcaseID']
giant_eucli.eucli_rank[giant_eucli['eucli_rank'] == 9223372036854775807] = 'invalid path'

In [80]:
merged_task1 = pd.merge(giant_query,giant_len, on=['uni', 'testcaseID'])

In [81]:
merged_task2_eucli = pd.merge(giant_query,giant_eucli, on=['uni', 'testcaseID'])

In [82]:
merged_task2 = pd.merge(merged_task2_eucli, giant_geo, on = ['uni', 'testcaseID'])

In [83]:
merged_test = pd.concat([merged_task1, merged_task2], axis=1)

In [87]:
for uni in uni_list:
    path = './grades/'
    name = uni
    task = '_task1'
    filename = path + name + task + '.csv'
    merged_task1[merged_task1.uni == uni].to_csv(filename, sep = ',')

In [71]:
'''
generate total score dataframe based on path length
@query_score
'''
from collections import defaultdict
length_score = defaultdict(float)
for testcaseID in range(1, 21):   
    print ("testcaseID: ", testcaseID)
    table_by_testcase = df_table[df_table['testcaseID'] == str(testcaseID)]
    table_ready = process_table(table_by_testcase, 'path_length')
    length_ranks = gen_rank(table_ready, 'path_length')
    length_performance = gen_performance_score(length_ranks)
    length_total_score = gen_total_score(length_performance)
    length_total_score['testcaseID'] = testcaseID
    if (testcaseID == 1):
        giant_len = length_total_score
    else:
        giant_len = [giant_len, length_total_score]
        giant_len = pd.concat(giant_len)
    for index, row in length_total_score.iterrows():
        key = frozenset({row['testcaseID'], row['uni']})
        length_score[key] = row['total_score']

testcaseID:  1
base_on:  path_length
testcaseID:  2
base_on:  path_length
testcaseID:  3
base_on:  path_length
testcaseID:  4
base_on:  path_length
testcaseID:  5
base_on:  path_length
testcaseID:  6
base_on:  path_length
testcaseID:  7
base_on:  path_length
testcaseID:  8
base_on:  path_length
testcaseID:  9
base_on:  path_length
testcaseID:  10
base_on:  path_length
testcaseID:  11
base_on:  path_length
testcaseID:  12
base_on:  path_length
testcaseID:  13
base_on:  path_length
testcaseID:  14
base_on:  path_length
testcaseID:  15
base_on:  path_length
testcaseID:  16
base_on:  path_length
testcaseID:  17
base_on:  path_length
testcaseID:  18
base_on:  path_length
testcaseID:  19
base_on:  path_length
testcaseID:  20
base_on:  path_length


In [72]:
'''
generate total score dataframe based on euclidian dist
@query_score
'''
from collections import defaultdict
eucli_score = defaultdict(float)
for testcaseID in range(21, 41):   
    print ("testcaseID: ", testcaseID)
    table_by_testcase = df_table[df_table['testcaseID'] == str(testcaseID)]
    table_ready = process_table(table_by_testcase, 'euclidean_dist')
    eucli_ranks = gen_rank(table_ready, 'euclidean_dist')
    eucli_performance = gen_performance_score(eucli_ranks)
    eucli_total_score = gen_total_score(eucli_performance)
    eucli_total_score['testcaseID'] = testcaseID
    if (testcaseID == 21):
        giant_eucli = eucli_total_score
    else:
        giant_eucli = [giant_eucli, eucli_total_score]
        giant_eucli = pd.concat(giant_eucli)
    for index, row in eucli_total_score.iterrows():
        key = frozenset({row['testcaseID'], row['uni']})
        eucli_score[key] = row['total_score']

testcaseID:  21
base_on:  euclidean_dist
testcaseID:  22
base_on:  euclidean_dist
testcaseID:  23
base_on:  euclidean_dist
testcaseID:  24
base_on:  euclidean_dist
testcaseID:  25
base_on:  euclidean_dist
testcaseID:  26
base_on:  euclidean_dist
testcaseID:  27
base_on:  euclidean_dist
testcaseID:  28
base_on:  euclidean_dist
testcaseID:  29
base_on:  euclidean_dist
testcaseID:  30
base_on:  euclidean_dist
testcaseID:  31
base_on:  euclidean_dist
testcaseID:  32
base_on:  euclidean_dist
testcaseID:  33
base_on:  euclidean_dist
testcaseID:  34
base_on:  euclidean_dist
testcaseID:  35
base_on:  euclidean_dist
testcaseID:  36
base_on:  euclidean_dist
testcaseID:  37
base_on:  euclidean_dist
testcaseID:  38
base_on:  euclidean_dist
testcaseID:  39
base_on:  euclidean_dist
testcaseID:  40
base_on:  euclidean_dist


In [73]:
'''
generate total score dataframe based on geo dist
@query_score
'''
from collections import defaultdict
geo_score = defaultdict(float)
for testcaseID in range(21, 41):   
    print ("testcaseID: ", testcaseID)
    table_by_testcase = df_table[df_table['testcaseID'] == str(testcaseID)]
    table_ready = process_table(table_by_testcase, 'geo_dist')
    geo_ranks = gen_rank(table_ready, 'geo_dist')
    geo_performance = gen_performance_score(geo_ranks)
    geo_total_score = gen_total_score(geo_performance)
    geo_total_score['testcaseID'] = testcaseID
    if (testcaseID == 21):
        giant_geo = geo_total_score
    else:
        giant_geo = [giant_geo,geo_total_score]
        giant_geo = pd.concat(giant_geo)
    for index, row in geo_total_score.iterrows():
        key = frozenset({row['testcaseID'], row['uni']})
        geo_score[key] = row['total_score']

testcaseID:  21
base_on:  geo_dist
testcaseID:  22
base_on:  geo_dist
testcaseID:  23
base_on:  geo_dist
testcaseID:  24
base_on:  geo_dist
testcaseID:  25
base_on:  geo_dist
testcaseID:  26
base_on:  geo_dist
testcaseID:  27
base_on:  geo_dist
testcaseID:  28
base_on:  geo_dist
testcaseID:  29
base_on:  geo_dist
testcaseID:  30
base_on:  geo_dist
testcaseID:  31
base_on:  geo_dist
testcaseID:  32
base_on:  geo_dist
testcaseID:  33
base_on:  geo_dist
testcaseID:  34
base_on:  geo_dist
testcaseID:  35
base_on:  geo_dist
testcaseID:  36
base_on:  geo_dist
testcaseID:  37
base_on:  geo_dist
testcaseID:  38
base_on:  geo_dist
testcaseID:  39
base_on:  geo_dist
testcaseID:  40
base_on:  geo_dist


In [47]:
'''
@final_score_map stores scores for task case 1-20
'''
final_score_map = defaultdict(float)
for testcaseID in range(1,21):
    for uni in uni_list:
        key = frozenset ({testcaseID, uni})
        final_score = query_score[key]*0.4 + length_score[key]*0.6
        #print ("key: ", key)
        #print ("length_score: ", length_score[key])
        #print ("query_score: ", query_score[key])
        final_score_map[key] = final_score

In [48]:
'''
@final_score_map adding scores for task case 21-40
'''
for testcaseID in range(21,41):
    for uni in uni_list:
        key = frozenset ({testcaseID, uni})
        final_score = query_score[key]*0.4 + max(geo_score[key], eucli_score[key])*0.6
        #print ("key: ", key)
        #print ("dist_score: ", max(geo_score[key], eucli_score[key]))
        #print ("query_score: ", query_score[key])
        final_score_map[key] = final_score

In [52]:
'''
sum over all testcase for each students
@final_grade
'''
final_grade = defaultdict(float)
for testcaseID in range(1,41):
    for uni in uni_list:
        key = frozenset ({testcaseID, uni})
        if (uni == 'pwh2126'):
            print ("id: ", testcaseID, final_score_map[key])
        final_grade[uni] += final_score_map[key]


id:  1 2.442857142857143
id:  2 2.4857142857142858
id:  3 2.3666666666666667
id:  4 2.1799999999999997
id:  5 2.065384615384615
id:  6 2.3041666666666663
id:  7 2.256666666666667
id:  8 0.0
id:  9 0.0
id:  10 2.194871794871795
id:  11 0.0
id:  12 2.1952380952380954
id:  13 0.0
id:  14 0.0
id:  15 2.2857142857142856
id:  16 0.0
id:  17 0.0
id:  18 0.0
id:  19 0.0
id:  20 0.0
id:  21 0.0
id:  22 0.0
id:  23 0.0
id:  24 0.0
id:  25 0.0
id:  26 0.0
id:  27 0.0
id:  28 0.0
id:  29 0.0
id:  30 0.0
id:  31 0.0
id:  32 0.0
id:  33 0.0
id:  34 0.0
id:  35 0.0
id:  36 0.0
id:  37 0.0
id:  38 0.0
id:  39 0.0
id:  40 0.0


In [31]:
'''
process table_by_testcase to overwrite nOfQueries for uni with invalid path,
in order to simulate the effect of dropping them outof the ranking
'''
def process_table (dataframe, based_on):
    if based_on == 'path_length': 
        table_by_testcase.path_length[table_by_testcase.is_valid != True] = -1 
    elif based_on == 'nOfQueries':
        table_by_testcase.nOfQueries[table_by_testcase.is_valid != True] = -1
    elif based_on == 'euclidean_dist':
        table_by_testcase.euclidean_dist[table_by_testcase.is_valid != True] = -1
    elif based_on == 'geo_dist':
        table_by_testcase.geo_dist[table_by_testcase.is_valid != True] = -1 
    table_ready = table_by_testcase[["uni","nOfQueries","path_length", "euclidean_dist", "geo_dist"]]
    return table_ready


In [32]:
'''
@gen_rank, given a table frame generate ranking of uni
'''
def gen_rank (dataframe, based_on):
    print ("base_on: ", based_on)
    ranks = []
    prev = sys.maxsize - 2
    #print ("min: ", prev)
    rank = 1
    dataframe[based_on] = dataframe[based_on].astype(float)
    dataframe = dataframe.sort_values(by = [based_on])
    for index, row in dataframe.iterrows():


        if (row[str(based_on)] == -1): #treat invalid entry as sys.maxsize

            ranks.append([row['uni'], sys.maxsize])
        else:
            if (row[str(based_on)] <= prev):
            
                ranks.append([row['uni'], rank])
            else:
            
                rank += 1
                ranks.append([row['uni'], rank])
            prev = row[str(based_on)]
            
    return ranks

In [33]:
'''
@gen_performance_score: based on ranking 
'''
def gen_performance_score(ranks):
    df_ranks = pd.DataFrame(ranks, columns = ['uni', 'rank'])
    min_rank = df_ranks['rank'][df_ranks['rank'] != sys.maxsize].min() #filter out invalid path records
    max_rank = df_ranks['rank'][df_ranks['rank'] != sys.maxsize].max()
    full_score = 0.5
    df_ranks['performance_score'] = df_ranks.apply(lambda row: ((max_rank - row['rank'] + 1)/max_rank)*full_score, axis=1)
    df_ranks.performance_score[df_ranks.performance_score < 0] = 0
    df_ranks['max_rank'] = max_rank
    return df_ranks

In [34]:
'''
@gen_total_score: add 2 to performance score if there is a valid path
'''
def gen_total_score(dataframe):
    dataframe['total_score'] = dataframe['performance_score'] + 2
    dataframe.total_score[dataframe.total_score == 2] = 0
    return dataframe